<h1>Importation des packages et des fonctions</h1>

<p>Parmi les packages importés on retrouve :<br>
   <ul>
       <li>fonctionsUtiles : package qui sert pour l'augmentation des données</li>
       <li>matplotlib, numpy : visualisation et manipulation de données</li>
       <li>cv2 : pour resize les images et les adapter à l'input du modèle</li> 
       <li>sklearn : pour la fonction train_test_split</li>
       <li>Tensorflow, keras : framework pour deeplearning</li>
   </ul>
   <b>Attention :</b> le package keras n'est pas celui d'origine, un patch non officiel a été appliqué pour régler un problème de BatchNormalization qui faisait s'écrouler les métriques sur les jeu de validation et de test, donnant alors l'illusion d'un problème d'overfitting.<br><br>
   Lien vers le git mentionnant le problème et la solution : https://github.com/keras-team/keras/pull/9965
</p>

In [1]:
import sys
sys.path.append('Autonomous_car/')

from fonctionsUtiles import functions as f
from fonctionsUtiles import architecture
from fonctionsUtiles import error

import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras import backend as K
tf.logging.set_verbosity(tf.logging.ERROR)

__init__


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h1>Chargement de toutes les données</h1>

<p>
    On récupère les données d'entrainement d'un projet (Axionaut) qui a gagné une des précédentes courses IronCar.
</p>

In [3]:
# Récupérer les données du circuit Axionable
X_axio = np.load('../Datasets/Classification/axionable_data/X_train_axio.npy')
Y_axio = np.load('../Datasets/Classification/axionable_data/Y_train_axio.npy')
print('Axionable data Loaded. Shape = ', np.shape(X_axio))

# Récupérer les données d'entrainement du circuit IronCar
# Nouveau circuit - Double chicane
X_chicane = np.load('../Datasets/Classification/ironcar_data/new_track/x_chicane.npy')
Y_chicane = np.load('../Datasets/Classification/ironcar_data/new_track/y_chicane.npy')
print('Ironcar new track chicane Loaded. Shape = ', np.shape(X_chicane))

# Ancien circuit - Dataset équilibré
X_iron = np.load('../Datasets/Classification/ironcar_data/old_track/balanced_iron_X.npy')
Y_iron = np.load('../Datasets/Classification/ironcar_data/old_track/balanced_iron_Y.npy')
print('Ironcar old track data Loaded. Shape = ', np.shape(X_iron))

Axionable data Loaded. Shape =  (26449, 90, 250, 3)
Ironcar new track chicane Loaded. Shape =  (1519, 90, 250, 3)
Ironcar old track data Loaded. Shape =  (16028, 90, 250, 3)


<h1>Resizing des données et tri des données non utilisables</h1>

<p>
    Pour l'approche Transfer Learning, il faut faire attention à l'input du modèle pré-entrainé que l'on utilise. Dans notre cas, MobileNetV2 admet en input des images de taille 224 x 224. Il faut donc resize les images qui sont de format 90 x 250
</p>

In [4]:
def resizing(array):
    masterX = list()
    for arr in array:
        masterX.append(cv2.resize(arr, (224,224)))
    return np.array(masterX)

X_axio_resized = resizing(X_axio)
X_chicane_resized = resizing(X_chicane)
X_iron_resized = resizing(X_iron)

print('Axionable data new size. Shape = ', np.shape(X_axio_resized))
print('Ironcar new track chicane new size. Shape = ', np.shape(X_chicane_resized))
print('Ironcar old track data new size. Shape = ', np.shape(X_iron_resized))

Axionable data new size. Shape =  (26449, 224, 224, 3)
Ironcar new track chicane new size. Shape =  (1519, 224, 224, 3)
Ironcar old track data new size. Shape =  (16028, 224, 224, 3)


<h1>Séparation des données en Train, Test et Validation</h1>

<p>
    On sépare le jeu de données en Training set, Validation set et Testing set selon les proportions suivantes :<br>
    <ul>
        <li>X_test = 20 % du set Xfinal_resized</li>
        <li>X_train = 80 % des données restantes de Xfinal_resized (soit 64 % du jeu total de données)</li>
        <li>X_val = 20 % des données restantes de Xfinal_resized (soit 16 % du jeu total de données)</li>
    </ul>
</p>

In [5]:
# Dictionnaire qui stock quelques variables importantes
args = {"augmentation": True,
        "train_split": 0.8,
        "val_split":0.2,
        "test_split":0.2,
        "early_stop":True,
       "patience":2}

<p>Avant d'effectuer la sépration du jeu de données en Train, Validation et Test, on concatène tous les vecteurs</p>

In [6]:
# Concatenation de toutes les données chargées
X = np.concatenate((X_axio_resized, X_chicane_resized, X_iron_resized))
Y = np.concatenate((Y_axio, Y_chicane, Y_iron))
print('All data loaded and concatenated. Shape = ', np.shape(X))

# Suppression des variables tmp qui contiennent les images
del X_axio, Y_axio, X_chicane, Y_chicane, X_iron, Y_iron, X_axio_resized, X_chicane_resized, X_iron_resized

All data loaded and concatenated. Shape =  (43996, 224, 224, 3)


In [7]:
# Réparatition du jeu de données en train test 80 - 20 %
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=12)

In [8]:
# Répartition du jeu d'entrainement en jeu d'entrainement et en jeu de validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=12)

In [9]:
print('All training data loaded and augmented. Shape = ', np.shape(X_train))
print('All validation data loaded and augmented. Shape = ', np.shape(X_val))
print('All testing data loaded and augmented. Shape = ', np.shape(X_test))

All training data loaded and augmented. Shape =  (28156, 224, 224, 3)
All validation data loaded and augmented. Shape =  (7040, 224, 224, 3)
All testing data loaded and augmented. Shape =  (8800, 224, 224, 3)


In [10]:
# Suppression des variables temporaires qui contiennent les images
del X,Y

<h1>Augmentation des données</h1>

<p>
    On augmente les données uniquement sur le jeu d'entrainement pour éviter que le jeu de validation et de test ne contiennent des données que le modèle aurait "déjà vu" pendant son entrainement.<br><br>
    On rajoute 10% d'image pour chacun des effets suivant :
    <ul>
        <li>luminosité augmentée</li>
        <li>effet de nuit</li>
        <li>flip horizontal</li>
        <li>ombres aléatoires sur la route</li>
        <li>les quatres transformations précédentes à la chaine</li>
    </ul>
</p>

In [10]:
# Data augmentation du dataset
if args['augmentation']:
    print('Augmenting data... Wait...')
    # Data augmentation 10% de luminosité aléatoire.
    X_bright, Y_bright = f.generate_brightness(X_train, Y_train, proportion=0.1)
    # Data augmentation 10% d'effet de nuit.
    X_night, Y_night = f.generate_night_effect(X_train, Y_train, proportion=0.1)
    # Data augmentation 10% de flip horizontal
    X_flip, Y_flip = f.generate_horizontal_flip(X_train, Y_train, proportion=0.1)
    # Data augmentation 10% d'ombres aléatoires.
    X_shadow, Y_shadow = f.generate_random_shadows(X_train, Y_train, proportion=0.1)
    # Data augmentation 10% de transformation à la chaine (bright + shadows + flip).
    X_chain, Y_chain = f.generate_chained_transformations(X_train, Y_train, proportion=0.1)

    # Concaténation des données générées avec les anciennes
    X_train = np.concatenate((X_train, X_bright, X_night,
                                 X_shadow))

    Y_train = np.concatenate((Y_train, Y_bright, Y_night, 
                                 Y_shadow)).astype('float32')

    print('Train data after augmentation. Shape = ', np.shape(X_train))

  2%|▏         | 46/2815 [00:00<00:06, 453.02it/s]

Augmenting data... Wait...


100%|██████████| 2815/2815 [00:04<00:00, 688.48it/s]


Train data after augmentation. Shape =  (36601, 224, 224, 3)


In [11]:
# Suppression des variables temporaires qui contiennent les images
del X_bright, Y_bright, X_night, Y_night, X_shadow, Y_shadow

<h1>Hyperparamètres</h1>

<p>
    On définit les hyperparamètres pour le modèle, ici le nombre de classes pour la classification (<code>num_classes</code>) la taille de l'input (<code>IMAGE_RES</code>), la taille des batch pour l'entrainement (<code>BATCH_SIZE</code>), et le nombre d'epoch pour la phase d'entrainement (<code>epochs</code>).
</p>

In [12]:
num_classes = 5
IMAGE_RES = 224
BATCH_SIZE = 40
epochs = 15

<h1>Définition du modèle</h1>

In [13]:
# Initialisation du modèle : on télécharge le modèle MobilNetV2 en ne prenant pas en compte la dernière couche car
# elle ne correspond pas à notre problème
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
l = Dense(128, activation = 'relu')(base_model.output)
l = Dropout(0.2)(l)
l = Dense(64, activation = 'relu')(l)
l = Dropout(0.2)(l)
l = Flatten()(l)
predictions = Dense(num_classes, activation = 'softmax')(l)
my_new_model = Model(inputs=base_model.input, outputs=predictions)

<p>
    Le réseau MobileNetV2 sans la dernière couche comporte 155 couches. Afin de respecter l'approche Transfer Learning fixée, on n'autorise pas l'entrainement pour ces 155 couches, au contraire de celles que l'on a rajouté par la suite.
</p>

In [14]:
for layer in my_new_model.layers[:154]:
   layer.trainable = False
 
for layer in my_new_model.layers[155:]:
   layer.trainable = True

In [15]:
# Cette cellule nous permet d'examiner le réseau obtenu et de vérifier que les layers que l'on veut figer l'ont bien étés
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [16]:
my_new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

<h1>Schéma du modèle</h1>

<p>
    La cellule suivante permet d'obtenir une représentation imagée du réseau obtenu
</p>

In [17]:
from keras.utils import plot_model
plot_model(my_new_model, to_file='model_class.png')

<h1>Entrainement du modèle</h1>

<p>
    On effectue l'entrainement selon les jeux de données d'entrainement et de validation définis plus haut dans le notebook. On utilise également les hyper paramètres définis auparavant.
</p>

In [ ]:
min_delta=.0005

#checkpoint pour sauver le modèle après chaque epoch
save_best = keras.callbacks.ModelCheckpoint(model_name, 
                                            monitor='val_loss', 
                                            verbose=1, 
                                            save_best_only=True, 
                                            mode='min')

#arrêt de l'entrainement dans le cas où la validation loss ne diminue plus
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                           min_delta=min_delta, 
                                           patience=args['patience'], 
                                           verbose=1, 
                                           mode='auto')
callbacks_list = [save_best]

if args['early_stop']:
    callbacks_list.append(early_stop)

hist = my_new_model.fit(
                X_train, 
                Y_train,
                epochs = epochs,
                batch_size=BATCH_SIZE, 
                verbose=1, 
                validation_data=(X_val, Y_val),
                callbacks=callbacks_list,
                shuffle=True)

model.save('TransferMobileNetV2_DoubleDenseLayer_15Epochs_class_keras_imp.h5')

Train on 36601 samples, validate on 7040 samples
Epoch 1/15
 2000/36601 [>.............................] - ETA: 21:48 - loss: 1.1517 - acc: 0.5490

<h1>Temps de prédiction pour une image</h1>

<p>
    La cellule suivante utilise une image du jeu de test et calcule le temps qu'il faut pour effectuer la prédiction.
</p>

In [ ]:
import time
start=time.time()
for i in range(1):
    my_new_model.predict(X_test[i-1:i])

print(time.time()-start)

<h1>Evaluation du modèle</h1>

<p>La cellule suivante permet de tester le modèle selon un jeu de test qu'il n'a jamais vu auparavant. On peut voir si le modèle a overfit les données ou non.</p>

In [ ]:
my_new_model.evaluate(X_test, Y_test)

<h1>Divers tracés</h1>

<h3>Evolution de l'accuracy train et val en fonction du nombre d'epoch</h3>

In [ ]:
plt.plot(hist.history['mean_squared_error'])
plt.plot(hist.history['val_mean_squared_error'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

<h3>Evolution de la loss train et val en fonction du nombre d'epoch</h3>

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

<h1>Chargement du modèle</h1>

<p>Les deux prochaines cellules servent à charger un modèle sauvegardé puis à l'évaluer sur un jeu de test. Attention cependant il faut redéfinir le jeux de test avant d'exécuter la deuxième cellule</p>

In [2]:
# K.clear_session()
# K.set_learning_phase(1)
my_new_model = load_model('TransferMobileNetV2_DoubleDenseLayer_15Epochs_class_keras_imp.h5')
my_new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [11]:
my_new_model.evaluate(X_test, Y_test)

8800/8800 [==============================] - 359s 41ms/step


[0.09493673544912863, 0.97625]

<p>
    La cellule suivante permet d'évaluer l'erreur du modèle (accuracy) parmi les catégories suivantes : forte gauche, gauche, tout droit, droite, forte droite.
</p>

In [12]:
Y_pred = my_new_model.predict(X_test)

Y_pred1=[0]*len(Y_pred)
for i in range(len(Y_pred)):
    Y_pred1[i]=Y_pred[i][0]
Y_pred1=np.array(Y_pred1)

error.error_classification(Y_test,Y_pred1)

ValueError: operands could not be broadcast together with shapes (8800,5) (8800,) 